In [ ]:
#Delete final_data.csv before run
import numpy as np 
import pandas as pd
data_raw = pd.read_csv('./DataSet.csv',delimiter=',', encoding = 'ISO-8859-1')
data_raw.head()
#Drop Original Resume
data = data_raw.drop('Resume', axis=1)
data.head()
#Đa dạng hóa dữ liệu
# Một item ở 1 category sẽ generate ra thêm 2 samples với cùng resume nhưng khác skill
# Skill sẽ được tổng hợp bởi những sample cùng loại 
# Sau đó nối ngẫu nhiên skill của n samples khác lại
from pandas import DataFrame
import random
labels = np.unique(data['Category'])
res = []
for i in labels:
    x = data[data['Category'] == i].values
    print(i, x.shape)
    # Tron data
    len = x.shape
    for j in range(0, len[0]):
        #tron n lan
        num = 2
        for c in range (0, num):
            rands = random.sample(range(0, len[0]), num)
            #[1,4,9]
            newSamples = x[j]
            d2 = ''
            for x1 in range(0, num):
                d2 += str(x[rands[x1]][2])
            newSamples[2] = d2
            x = np.vstack([x, newSamples])

    if res.__len__() == 0:
        res = x
    else:
        res = np.vstack([res, x])
print(res.shape)
df = DataFrame(
    {'Category': res[:, 0], 'Resume': res[:, 1], 'Skill': res[:, 2]})
df.to_csv('final_data.csv',  index=False)


In [ ]:
#%% Import libraries
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
import re
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
np.set_printoptions(precision=4)

nltk.download('stopwords')
nltk.download('punkt')

#%% Load data
data = pd.read_csv('./final_data.csv', delimiter=',', encoding = 'ISO-8859-1')
data.head()
#%%Bar graph visualization
plt.figure(figsize=(15,15))
plt.xticks(rotation=90)
sns.countplot(y="Category", data=data)



In [ ]:
# %% Get set of stopwords
#stopwords_set = set(stopwords.words('english')+['``',"''"])

gist_file = open("./stopword.txt", "r")
try:
    content = gist_file.read()
    stopwords_set = content.split(",")
finally:
    gist_file.close()
stopwords_set = set(stopwords_set)

# %% Function to clean resume text
def clean_text(resume_text):
    try:
      resume_text = re.sub('http\S+\s*', ' ', resume_text) 
      resume_text = re.sub('RT|cc', ' ', resume_text) 
      resume_text = re.sub('#\S+', '', resume_text) 
      resume_text = re.sub('@\S+', '  ', resume_text) 
      resume_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resume_text) 
      resume_text = re.sub(r'[^\x00-\x7f]',r' ', resume_text) 
      resume_text = re.sub('\s+', ' ', resume_text) 
      resume_text = resume_text.lower()  
      resume_text_tokens = word_tokenize(resume_text) 
      filtered_text = [w for w in resume_text_tokens if not w in stopwords_set] 
      return ' '.join(filtered_text)
    except:
      return ''
    

# %% Print a sample original resume
print('--- Original resume ---')
#print(data['Resume'][0])
data['cleaned_resume'] = data.Resume.apply(lambda x: clean_text(x))
data['cleaned_skill'] = data.Skill.apply(lambda x: clean_text(x))

print('--- Cleaned resume ---')
#print(data['cleaned_resume'][0])


In [ ]:
print(data['cleaned_skill'][0])

In [ ]:

# %%Get features and labels from data and shuffle
features1 = data['cleaned_resume'].values
features2 = data['cleaned_skill'].values

original_labels = data['Category'].values
labels = original_labels[:]
data_size = data.__len__()
for i in range(data_size):
  labels[i] = str(str(labels[i]).lower())
  labels[i] = labels[i].replace(" ", "") 
  labels[i] = labels[i].replace("-", "") 

features1,features2, labels = shuffle(features1,features2, labels)


# %% Split for train and test
train_split = 0.75
train_size = int(train_split * data_size)

train_features = [features1[:train_size],features2[:train_size]]
train_labels = labels[:train_size]

test_features = [features1[train_size:],features2[train_size:]]
test_labels = labels[train_size:]

# Print size of each split
print(train_labels.__len__())
print(test_labels.__len__())



In [ ]:

# %%Tokenize feature data and print word dictionary
vocab_size = 6000
oov_tok = '<OOV>'
feature_tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
feature_tokenizer.fit_on_texts(features1)
feature_tokenizer.fit_on_texts(features2)
feature_index = feature_tokenizer.word_index
# Print example sequences from train and test datasets
train_feature_sequences = [feature_tokenizer.texts_to_sequences(train_features[0]),feature_tokenizer.texts_to_sequences(train_features[1])]
test_feature_sequences = [feature_tokenizer.texts_to_sequences(test_features[0]),feature_tokenizer.texts_to_sequences(test_features[1])]


In [ ]:

# %% Tokenize label data and print label dictionary
label_tokenizer = Tokenizer(lower=True)
label_tokenizer.fit_on_texts(labels)
label_index = label_tokenizer.word_index
print(dict(list(label_index.items())))
# Print example label encodings from train and test datasets
train_label_sequences = label_tokenizer.texts_to_sequences(train_labels)
test_label_sequences = label_tokenizer.texts_to_sequences(test_labels)


# %%Pad sequences for feature data
max_length = 1200
trunc_type = 'post'
pad_type = 'post'

train_feature_padded = np.array([pad_sequences(train_feature_sequences[0], maxlen=max_length, padding=pad_type, truncating=trunc_type),pad_sequences(train_feature_sequences[1], maxlen=max_length, padding=pad_type, truncating=trunc_type)])
test_feature_padded = np.array([pad_sequences(test_feature_sequences[0], maxlen=max_length, padding=pad_type, truncating=trunc_type),pad_sequences(test_feature_sequences[1], maxlen=max_length, padding=pad_type, truncating=trunc_type)])

embedding_dim = 64
units = np.unique(data['Category']).__len__() + 1

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, Flatten, Dense

# First input
input1_weight = 0.6
input2_weight = 0.4
input1 = Input(shape=(max_length,), name='input1')
x1 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(input1)*input1_weight

# Second input
input2 = Input(shape=(max_length,), name='input2')
x2 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(input2)*input2_weight

# Concatenate embeddings
x = Concatenate()([Flatten()(x1), Flatten()(x2)])

# Add more layers to the model
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(units, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs=[input1, input2], outputs=output)

#%% # Compile the model and convert train/test data into NumPy arrays
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train_feature_padded = np.array(train_feature_padded)
# test_feature_padded = np.array(test_feature_padded)
train_label_sequences = np.array(train_label_sequences)
test_label_sequences = np.array(test_label_sequences)

# %%
num_epochs = 100
model.summary()


In [ ]:
history = model.fit((train_feature_padded[0],train_feature_padded[1]), train_label_sequences, epochs=num_epochs, validation_data=((test_feature_padded[0],test_feature_padded[1]), test_label_sequences), verbose=2)
score = model.evaluate((test_feature_padded[0],test_feature_padded[1]), test_label_sequences, verbose=1)
print("Test Accuracy:", score[1])

#%% Draw accuracy model & loss model
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# %%
model.save('models_P')

# %% Save feature tokenizer
with open('feature_tokenizer.pickle', 'wb') as handle:
    pickle.dump(feature_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# %% Save reverse dictionary of labels to encodings
label_to_encoding = dict(list(label_index.items()))

encoding_to_label = {}
for k, v in label_to_encoding.items():
  encoding_to_label[v] = k
print(encoding_to_label)
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(encoding_to_label, handle, protocol=pickle.HIGHEST_PROTOCOL)





In [ ]:
# %% Create padded sequence for example
input = """Work History      August 2010  -  Current    Teacher   |   Company Name   |   City  ,   State     Executed lesson plans and evaluated the effectiveness through assessment and reflection Created visual supports, using technology to enhance and to differentiate instruction Use the Go Math, Eureka, & Engage NY, IXL & Khan Academy resources to provide math instruction Lead inquiry-based student investigations for the FOSS Science Program using Next Generation Standards Work with the Common Core & Next Generation Standards to provide structured instruction Provide Smarter Balanced NWEA Test Preparation for Mathematics & Science Create instructional materials and strategies consistent with student learning and behavioral needs Utilize the Positive Behavior Support (PBS) Program to provide positive reinforcement as a preventive measure for disruptive and/or inappropriate behavior Collaborate with grade partners to provide consistent instruction, and to develop and implement grade-level goals Update a teacher infinite campus with homework, grades and events to extend parental and student communication.         August 2008  -  June 2010    6th Grade Teacher   |   Company Name   |   City  ,   State     Provide a balanced literacy program to fifth graders based on Shelton's curriculum goal of the whole, small, whole Reader's Workshop Model, in order to increase student growth in reading stamina and comprehension skills Use various strategies and procedures for small group instruction to increase student growth in vocabulary, fluency, and comprehension Utilize Technology across multiple content areas Teach students strategies for previewing books and choose a book that is "just-right" Provide Writing Instruction through the Writer's Workshop model Utilize the Houghton Mifflin Anthology to model various meta-cognitive reading strategies Supplement the curriculum with various non-fiction texts Utilize Literacy Work Stations during small group instruction time, such as an Independent Daily Reading Station, Word Study Station, and a CMT Strand Work Station Provide small group instruction based on analysis of weak CMT strands, as well as skill needs in the subjects of Reading, Writing, and Math Administer and analyze the Developmental Reading Assessment (DRA 2) Provide application lessons to improve students' Degrees of Reading Power skills on the Connecticut Mastery Tests Used the Everyday Mathematics Curriculum to provide math instruction Lead inquiry-based student investigations for the FOSS Science Program Provide Connecticut Mastery Test Preparation for Reading, Writing, Mathematics, and Science Create instructional materials and strategies consistent with student learning and behavioral needs Utilize the Positive Behavior Support (PBS) Program to provide positive reinforcement as a preventive measure for disruptive and/or inappropriate behavior.         September 2007  -  June 2008    Special Education Tutor   |   Company Name   |   City  ,   State     Modify math curriculum and created worksheets for a sixth grade student in accordance with IEP objectives Create accommodated quizzes, tests, lesson and study guides for sixth grade Social Studies curriculum Provide small group instruction for core content subjects in K-6 general education classrooms Provide assistance to general education teacher using the Inclusion Model for instruction Assess Kindergarten students using Shelton Public Schools Assessment Administer the Brigance Inventory of Basic Skills to special education students Administer CMT's to special education students Provide accommodations during sixth grade Writer's Workshop for a group of six students Planned and implemented an introduction to the 6th grade Science Embedded Task utilizing the Scientific Method Create visual supports using technology to enhance and to differentiate instruction Observe and assist with Everyday Mathematics, Wilson Reading, Houghton Mifflin, Scott Foresman Social Studies lesson, CMT Prep, and Literature Circles.         Education     2003   Diploma  :       Derby High School  ,   City  ,   State         2007   NCATE Endorsement, National Council for Accreditation of Teacher Education Connecticut Collaboration (Elementary Regular Education/Special Education) (013, 165 certifications)  :       Southern Connecticut State University  ,   City  ,   State         2010   Collaborative Endorsement (Elementary Regular/Special Education) (013, 165)  :       Southern Connecticut State University  ,   City  ,   State   NCATE Endorsement, National Council for Accreditation of Teacher Education Remedial Reading and Remedial Language Arts, Grades K-12 (102 Certification) GPA: 3.85 Magna Cum Laude, Outstanding Special Education Senior Award      May 2007   Bachelor of Arts  :   Psychology   Psychology         Passed Praxis II (Regular Education) *Passed Praxis II (Special Education) *Remedial Reading and Remedial Language Arts, Grades K-12 (102 Certification) *Passed Reading Test  :             Skills    Arts, Basic, book, content, Council, in K, Inspiration, instruction, Internet Browsers, Inventory, lesson plans, materials, Math, Mathematics, Excel, Microsoft Office, PowerPoint, Publisher, Microsoft Windows XP Professional, Word, Next, non-fiction, Speaking, Reading, Scientific, structured, Teacher, Writer      Additional Information      MEMBERSHIPS/HONORS: Academic Dean's List at Southern Connecticut State University  - (Fall 2003, Fall 2004, Fall 2005, Spring 2006, Fall 2006) Psi-Chi psychology national honor society Golden Key national honor society Kappa Delta Pi national honor society Most Outstanding Special Education Senior"""
resume_example = clean_text(input)
example_sequence = feature_tokenizer.texts_to_sequences([resume_example])
example_padded = pad_sequences(example_sequence, maxlen=max_length, padding=pad_type, truncating=trunc_type)
example_padded = np.array(example_padded)

input2 = """TEACHER         Professional Summary    An experienced human resource training professional with demonstrated success in developing, delivering and evaluating, corporate training programs, 2+ years of work with newly developed tools for rapid e-learning development. Special skills in online training for a variety of audiences. Recognized for alignment of training solutions with business goals, management of project and people, process improvement, needs analysis and training evaluation.      Core Qualifications          Microsoft Words  Outlook  Internet  PowerPoint  ADDIE MODEL  Microsoft Office  Adobe Photoshop  Audacity Sound Booth  Moodle  AdobeCS5  Captivate  Adobe Premier  Flash  Blackboard 9.1  PeopleSoft"""
resume_example2 = clean_text(input2)
example_sequence2 = feature_tokenizer.texts_to_sequences([resume_example2])
example_padded2 = pad_sequences(example_sequence2, maxlen=max_length, padding=pad_type, truncating=trunc_type)
example_padded2 = np.array(example_padded2)
#print(example_padded)
# Make a prediction
print((example_padded,example_padded2))
prediction = model.predict((example_padded,example_padded2))
# Verify that prediction has correct format
# print(prediction[0])
# print(np.sum(prediction[0])) 

# Indices of top 5 most probable solutions
indices = np.argpartition(prediction[0], -5)[-5:]
indices = indices[np.argsort(prediction[0][indices])]
indices = list(reversed(indices))
print(indices)

# Find maximum value in prediction and its index
#print(prediction[0])
# print(max(prediction[0])) 
# print(np.argmax(prediction[0])) 

# 
for index in indices:
    print(prediction[0][index]*100,"% ",encoding_to_label[index])

# %%